In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout, MaxPooling2D, BatchNormalization, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import Callback, EarlyStopping
from sklearn.model_selection import train_test_split
import time

from tensorflow_privacy.privacy.analysis.rdp_accountant import compute_rdp
from tensorflow_privacy.privacy.analysis.rdp_accountant import get_privacy_spent
from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasAdamOptimizer
from tensorflow_privacy.privacy.analysis.compute_noise_from_budget_lib import compute_noise


from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack import membership_inference_attack as mia
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackType

import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.plotting as plotting

import numpy as np
from scipy import special

import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
learning_rate = 15e-5

In [8]:
def load_mnist():
    """Loads MNIST and preprocesses to combine training and validation data."""
    train, test = tf.keras.datasets.mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    train_data = train_data.reshape((train_data.shape[0], 28, 28, 1))
    test_data = test_data.reshape((test_data.shape[0], 28, 28, 1))

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
    test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

    assert train_data.min() == 0.
    assert train_data.max() == 1.
    assert test_data.min() == 0.
    assert test_data.max() == 1.

    return train_data, train_labels, test_data, test_labels

def membership_inference_attack(model, X_train, X_test, y_train, y_test):
    print('Predict on train...')
    logits_train = model.predict(X_train, batch_size=batch_size)
    print('Predict on test...')
    logits_test = model.predict(X_test, batch_size=batch_size)

    print('Apply softmax to get probabilities from logits...')
    prob_train = special.softmax(logits_train, axis=1)
    prob_test = special.softmax(logits_test, axis=1)

    print('Compute losses...')
    cce = tf.keras.backend.categorical_crossentropy
    constant = tf.keras.backend.constant

    loss_train = cce(constant(y_train), constant(prob_train), from_logits=False).numpy()
    loss_test = cce(constant(y_test), constant(prob_test), from_logits=False).numpy()
    
    labels_train = np.argmax(y_train, axis=1)
    labels_test = np.argmax(y_test, axis=1)

    input = AttackInputData(
      logits_train = logits_train,
      logits_test = logits_test,
      loss_train = loss_train,
      loss_test = loss_test,
      labels_train = labels_train,
      labels_test = labels_test
    )

    # Run several attacks for different data slices
    attacks_result = mia.run_attacks(input,
                                     SlicingSpec(
                                         entire_dataset = True,
                                         by_class = True,
                                         by_classification_correctness = True
                                     ),
                                     attack_types = [
                                         AttackType.THRESHOLD_ATTACK,
                                         AttackType.LOGISTIC_REGRESSION,
                                         AttackType.MULTI_LAYERED_PERCEPTRON,
                                         AttackType.RANDOM_FOREST, 
                                         AttackType.K_NEAREST_NEIGHBORS,
                                         AttackType.THRESHOLD_ENTROPY_ATTACK
                                     ])

    # Plot the ROC curve of the best classifier
#     fig = plotting.plot_roc_curve(
#         attacks_result.get_result_with_max_auc().roc_curve)

    # Print a user-friendly summary of the attacks
    print(attacks_result.summary(by_slices = True))
    time.sleep(5)
    return attacks_result.get_result_with_max_auc().get_auc(), attacks_result.get_result_with_max_attacker_advantage().get_attacker_advantage()

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [9]:
def create_cnn(dropout=None, regularizer=None):
    input_data = Input(shape = X_train[0].shape)
    x = Reshape(target_shape=(28, 28, 1))(input_data)
    x = Conv2D(filters=12, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    if dropout is not None:
        x = Dropout(dropout)(x)
    output = Dense(10, kernel_regularizer=regularizer)(x)

    model = Model(input_data, output)
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
            
    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
    
    return model

def create_dp_cnn(noise_multiplier, l2_norm_clip, microbatches):
    input_data = Input(shape = X_train[0].shape)
    x = Reshape(target_shape=(28, 28, 1))(input_data)
    x = Conv2D(filters=12, kernel_size=(3, 3), activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2))(x)
    x = Flatten()(x)
    output = Dense(10)(x)

    model = Model(input_data, output)
    
    optimizer = DPKerasAdamOptimizer(
                            l2_norm_clip=l2_norm_clip,
                            noise_multiplier=noise_multiplier,
                            num_microbatches=microbatches,
                            learning_rate=learning_rate)
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True, reduction=tf.losses.Reduction.NONE)
    
    model.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])
    
    return model

In [10]:
X_train, y_train, X_test, y_test = load_mnist()
X_train_flat, X_test_flat = X_train.reshape(-1, 28*28*1), X_test.reshape(-1, 28*28*1)

In [6]:
epochs = 50
batch_size = 100
attacks = 1
settings = [
    (None,None),
    (0.25,None),
    (0.50,None),
    (0.75,None),
    (None,'l2'),
    (0.25,'l2'),
    (0.50,'l2'),
    (0.75,'l2'),
]
results_summary = []

for drop, reg in settings:
    # Instantiate network
    model = create_cnn(dropout=drop, regularizer=reg)
    
    # Train network until convergence
    start_time = time.time()
    r = model.fit(X_train, 
                y_train, 
                validation_data=(X_test, y_test),
                epochs=epochs, 
                batch_size=batch_size,
                #callbacks=[callback]
               )
    end_time = time.time()
    time_elapsed = (end_time - start_time)

    # MIA 
    aauc = []
    aadv = []
    for _ in range(attacks):
        auc, adv = membership_inference_attack(model, X_train, X_test, y_train, y_test)
        aauc.append(auc)
        aadv.append(adv)
    mauc = sum(aauc) / attacks
    madv = sum(aadv) / attacks

    # Write result summary
    summ = ', '.join(map(str,[
        len(r.history['loss']), #epochs
        drop,
        reg,
        r.history['loss'][-1], 
        r.history['val_loss'][-1],
        r.history['accuracy'][-1],
        r.history['val_accuracy'][-1],
        time_elapsed,
        mauc,
        madv
    ]))

    results_summary.append(summ)
    print('='*40)
    
print('Epochs, Dropout, Regularizer, Loss, Val loss, Accuracy, Val accuracy, Time, AUC, Advantage')
for r in results_summary:
    print(r)

Epoch 1/50
600/600 [==============================] - 9s 3ms/step - loss: 0.3917 - accuracy: 0.8974 - val_loss: 0.1666 - val_accuracy: 0.9534
Epoch 2/50
600/600 [==============================] - 1s 2ms/step - loss: 0.1394 - accuracy: 0.9601 - val_loss: 0.1084 - val_accuracy: 0.9690
Epoch 3/50
547/600 [==========================>...] - ETA: 0s - loss: 0.1020 - accuracy: 0.9710 ETA: 0s - loss:

KeyboardInterrupt: 

In [39]:
# TEST FOR DIFFERENT EPS
results_summary = []

n = X_train.shape[0]
epochs = 50
batch_size = 100
microbatches = 100
#epsilons = [0.1,0.5,1,2,4,8,16,100,1000]
epsilons = np.linspace(0.1, np.log(2), 5) # 5 values from 0.1 to ln2
#delta = 1e-6
delta = 1e-100 # very small delta
min_noise = 1e-100
l2_norm_clip = 2.5
sampling_rate = batch_size / n
attacks = 1

for e in epsilons:
    # Compute noise multiplier from target epsilon
    noise_multiplier = compute_noise(n, batch_size, e, epochs, delta, min_noise)
    
    # Compute epsilon
    orders = [1 + x / 10. for x in range(1, 100)] + list(range(11, 101))
    sampling_probability = batch_size / n
    rdp = compute_rdp(q=sampling_probability,
                    noise_multiplier=noise_multiplier,
                    steps=epochs * n // batch_size,
                    orders=orders)
    eps = get_privacy_spent(orders, rdp, target_delta=delta)
    
    # Instantiate network
    model = create_dp_cnn(noise_multiplier, l2_norm_clip, microbatches)

    # Train network
    start_time = time.time()
    r = model.fit(X_train, 
                 y_train, 
                 validation_data=(X_test, y_test), 
                 epochs=epochs, 
                 batch_size=batch_size,
                # callbacks=[callback]
                )
    end_time = time.time()
    time_elapsed = (end_time - start_time)

    # MIA
    aauc = []
    aadv = []
    for _ in range(attacks):
        auc, adv = membership_inference_attack(model, X_train, X_test, y_train, y_test)
        aauc.append(auc)
        aadv.append(adv)
    mauc = sum(aauc) / attacks
    madv = sum(aadv) / attacks

    # Write result summary
    summ = ', '.join(map(str,[
          len(r.history['loss']),
          e,
          delta,
          l2_norm_clip,
          noise_multiplier,
          sampling_rate,
          eps[0],
          r.history['loss'][-1], 
          r.history['val_loss'][-1],
          r.history['accuracy'][-1],
          r.history['val_accuracy'][-1],
          time_elapsed,
          mauc,
          madv
    ]))
    results_summary.append(summ)
    print('='*40)
    
print('Epochs, Target epsilon, delta, C, Sigma, Sampling rate, Epsilon, Loss, Val loss, Accuracy, Val accuracy, Time, AUC, Advantage')
for r in results_summary:
    print(r)

DP-SGD with sampling rate = 0.167% and noise_multiplier = 13802.88224639123 iterated over 30000 steps satisfies differential privacy with eps = 0.1 and delta = 1e-100.
Epoch 1/50
600/600 [==============================] - 46s 76ms/step - loss: 2.3116 - accuracy: 0.1164 - val_loss: 2.3100 - val_accuracy: 0.1190
Epoch 2/50
600/600 [==============================] - 46s 76ms/step - loss: 2.3126 - accuracy: 0.1156 - val_loss: 2.3107 - val_accuracy: 0.1153
Epoch 3/50
600/600 [==============================] - 47s 78ms/step - loss: 2.3098 - accuracy: 0.1142 - val_loss: 2.3073 - val_accuracy: 0.1196
Epoch 4/50
600/600 [==============================] - 46s 76ms/step - loss: 2.3064 - accuracy: 0.1141 - val_loss: 2.3038 - val_accuracy: 0.1084
Epoch 5/50
600/600 [==============================] - 45s 76ms/step - loss: 2.3037 - accuracy: 0.1094 - val_loss: 2.3014 - val_accuracy: 0.1089
Epoch 6/50
600/600 [==============================] - 49s 81ms/step - loss: 2.3047 - accuracy: 0.1029 - val_loss

C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iter

Best-performing attacks over all slices
  RANDOM_FOREST (with 1010 training and 1010 test examples) achieved an AUC of 0.58 on slice CLASS=3
  MULTI_LAYERED_PERCEPTRON (with 980 training and 980 test examples) achieved an advantage of 0.15 on slice CLASS=0

Best-performing attacks over slice: "Entire dataset"
  MULTI_LAYERED_PERCEPTRON (with 10000 training and 10000 test examples) achieved an AUC of 0.54
  MULTI_LAYERED_PERCEPTRON (with 10000 training and 10000 test examples) achieved an advantage of 0.07

Best-performing attacks over slice: "CLASS=0"
  MULTI_LAYERED_PERCEPTRON (with 980 training and 980 test examples) achieved an AUC of 0.56
  MULTI_LAYERED_PERCEPTRON (with 980 training and 980 test examples) achieved an advantage of 0.15

Best-performing attacks over slice: "CLASS=1"
  LOGISTIC_REGRESSION (with 1135 training and 1135 test examples) achieved an AUC of 0.53
  MULTI_LAYERED_PERCEPTRON (with 1135 training and 1135 test examples) achieved an advantage of 0.07

Best-perfor

600/600 [==============================] - 45s 75ms/step - loss: 2.2984 - accuracy: 0.0856 - val_loss: 2.3033 - val_accuracy: 0.0795
Epoch 34/50
600/600 [==============================] - 45s 75ms/step - loss: 2.2977 - accuracy: 0.0910 - val_loss: 2.3006 - val_accuracy: 0.0817
Epoch 35/50
600/600 [==============================] - 45s 76ms/step - loss: 2.2965 - accuracy: 0.0904 - val_loss: 2.3005 - val_accuracy: 0.0885
Epoch 36/50
600/600 [==============================] - 45s 75ms/step - loss: 2.2963 - accuracy: 0.0908 - val_loss: 2.3022 - val_accuracy: 0.0816
Epoch 37/50
600/600 [==============================] - 45s 75ms/step - loss: 2.2949 - accuracy: 0.0897 - val_loss: 2.2974 - val_accuracy: 0.0907
Epoch 38/50
600/600 [==============================] - 45s 75ms/step - loss: 2.2930 - accuracy: 0.0956 - val_loss: 2.2964 - val_accuracy: 0.0963
Epoch 39/50
600/600 [==============================] - 45s 75ms/step - loss: 2.2922 - accuracy: 0.0979 - val_loss: 2.2976 - val_accuracy: 0.09

C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#l

...

C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario

Best-performing attacks over all slices
  LOGISTIC_REGRESSION (with 1009 training and 1009 test examples) achieved an AUC of 0.62 on slice CLASS=9
  MULTI_LAYERED_PERCEPTRON (with 1010 training and 1010 test examples) achieved an advantage of 0.22 on slice CLASS=3

Best-performing attacks over slice: "Entire dataset"
  K_NEAREST_NEIGHBORS (with 10000 training and 10000 test examples) achieved an AUC of 0.54
  K_NEAREST_NEIGHBORS (with 10000 training and 10000 test examples) achieved an advantage of 0.07

Best-performing attacks over slice: "CLASS=0"
  RANDOM_FOREST (with 980 training and 980 test examples) achieved an AUC of 0.59
  RANDOM_FOREST (with 980 training and 980 test examples) achieved an advantage of 0.17

Best-performing attacks over slice: "CLASS=1"
  LOGISTIC_REGRESSION (with 1135 training and 1135 test examples) achieved an AUC of 0.53
  LOGISTIC_REGRESSION (with 1135 training and 1135 test examples) achieved an advantage of 0.09

Best-performing attacks over slice: "CLA

600/600 [==============================] - 47s 78ms/step - loss: 0.5315 - accuracy: 0.8318 - val_loss: 0.4988 - val_accuracy: 0.8441
Epoch 34/50
600/600 [==============================] - 48s 80ms/step - loss: 0.5240 - accuracy: 0.8346 - val_loss: 0.4913 - val_accuracy: 0.8473
Epoch 35/50
600/600 [==============================] - 47s 79ms/step - loss: 0.5168 - accuracy: 0.8371 - val_loss: 0.4849 - val_accuracy: 0.8498
Epoch 36/50
600/600 [==============================] - 48s 79ms/step - loss: 0.5116 - accuracy: 0.8386 - val_loss: 0.4795 - val_accuracy: 0.8508
Epoch 37/50
600/600 [==============================] - 48s 79ms/step - loss: 0.5050 - accuracy: 0.8409 - val_loss: 0.4733 - val_accuracy: 0.8532
Epoch 38/50
600/600 [==============================] - 47s 79ms/step - loss: 0.4993 - accuracy: 0.8424 - val_loss: 0.4668 - val_accuracy: 0.8549
Epoch 39/50
600/600 [==============================] - 47s 79ms/step - loss: 0.4948 - accuracy: 0.8440 - val_loss: 0.4612 - val_accuracy: 0.85

C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Usuario

Best-performing attacks over all slices
  RANDOM_FOREST (with 1028 training and 1028 test examples) achieved an AUC of 0.62 on slice CLASS=7
  MULTI_LAYERED_PERCEPTRON (with 958 training and 958 test examples) achieved an advantage of 0.20 on slice CLASS=6

Best-performing attacks over slice: "Entire dataset"
  RANDOM_FOREST (with 10000 training and 10000 test examples) achieved an AUC of 0.57
  RANDOM_FOREST (with 10000 training and 10000 test examples) achieved an advantage of 0.11

Best-performing attacks over slice: "CLASS=0"
  RANDOM_FOREST (with 980 training and 980 test examples) achieved an AUC of 0.55
  RANDOM_FOREST (with 980 training and 980 test examples) achieved an advantage of 0.10

Best-performing attacks over slice: "CLASS=1"
  RANDOM_FOREST (with 1135 training and 1135 test examples) achieved an AUC of 0.55
  RANDOM_FOREST (with 1135 training and 1135 test examples) achieved an advantage of 0.11

Best-performing attacks over slice: "CLASS=2"
  RANDOM_FOREST (with 1032